In [ ]:
import os
from dotenv import load_dotenv
import torchvision
from torchvision import transforms
import torch
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
from helpers.dataset import CustomDataset
from helpers.model import CNNImageClassification, fit
from helpers.helper import show_batch

In [ ]:
load_dotenv()

roboflow_key = os.getenv('roboflow_key')

# Download dataset

In [ ]:
#!curl -L "https://universe.roboflow.com/ds/WFOxHVrIFE?key={roboflow_key}" > roboflow.zip; unzip roboflow.zip -d data; rm roboflow.zip

In [ ]:
data_dir = "data/train"
val_data_dir = "data/valid"
test_data_dir = "data/test"

data_csv = 'data/train/_classes.csv'
val_data_csv = 'data/valid/_classes.csv'
test_data_csv = 'data/test/_classes.csv'

In [ ]:
train_data = CustomDataset(data_csv, data_dir, transform = transforms.Compose([
    transforms.Resize((224,224)),transforms.ToTensor()
]))

val_data = CustomDataset(val_data_csv, val_data_dir, transform = transforms.Compose([
    transforms.Resize((224,224)),transforms.ToTensor()
]))

test_data = CustomDataset(test_data_csv, test_data_dir, transform = transforms.Compose([
    transforms.Resize((224,224)),transforms.ToTensor()
]))

In [ ]:
train_data

In [ ]:
data, label = train_data[0]

In [ ]:
data

In [ ]:
label

In [ ]:
batch_size = 128

In [ ]:
train_dl = DataLoader(train_data, batch_size, shuffle = True, num_workers = 8, pin_memory = True)
val_dl = DataLoader(val_data, batch_size, shuffle = True, num_workers = 8, pin_memory = True)
test_dl = DataLoader(test_data, batch_size, shuffle = True, num_workers = 8, pin_memory = True)

In [ ]:
show_batch(train_dl)

In [ ]:
model = CNNImageClassification()

In [ ]:
num_epochs = 30
opt_func = torch.optim.Adam
lr = 0.001
#fitting the model on training data and record the result after each epoch
history = fit(num_epochs, lr, model, train_dl, val_dl, opt_func)